In [ ]:
from folium.plugins import HeatMap
!pip install scipy numpy matplotlib pandas sklearn > /dev/null
!pip install requests
!pip install geopandas
!pip install geopy
!pip install folium

In [ ]:
# Load libraries
import json
import requests
import pandas as pd
from pandas.io.json import json_normalize
from geopy import Nominatim
import numpy as np
import webbrowser
import folium

<h2>Dataset load</h2>

In [ ]:
                  
def get_jobs_info(api_url = 'https://justjoin.it/api/offers'):
    headers = {'Content-Type': 'application/json'}
    response = requests.get(api_url, headers=headers)
    if response.status_code == 200:
        return json.loads(response.content.decode('utf-8'))
    else:
        return None

<h2>Coordinates load and HeatMap </h2>

In [ ]:
# Columns name used in building HeatMap
lat_column_name = 'lat'
lon_column_name = 'lon'
numerical_column_name = 'no'
    
def fill_geo_info(places_data, loc_column_name):
    locator = Nominatim(user_agent="myGeocoder")
    
    for i, row in places_data.iterrows():
        location = locator.geocode(row[loc_column_name])
        if location is not None:
            places_data.loc[i,lat_column_name] = location.latitude
            places_data.loc[i,lon_column_name] = location.longitude
            
    return places_data


def print_geo_map(places_data, file_path = 'test.html'):

    places_len = len(places_data)
    
    lat = np.array(places_data[lat_column_name][0:places_len])
    lon = np.array(places_data[lon_column_name][0:places_len])
    no = np.array(places_data[numerical_column_name][0:places_len],dtype=float)
    data1 = [[lat[i],lon[i],no[i]] for i in range(places_len)] 
    
    #location is the center location, draw a Map, and start zooming is 6 times.
    map_osm = folium.Map(location=[lat.mean(),lon.mean()],zoom_start=6,control_scale=True)
    HeatMap(data1).add_to(map_osm) # Add heat map to the created map
    map_osm.save(file_path) # Save as html file
    webbrowser.open(file_path) # Default browser open

In [ ]:
def group_by_city(data, address_column_name, id_column_name = 'id'):
    unique_places = data.groupby(address_column_name)[id_column_name].nunique()
    places = pd.DataFrame({city_column_name:unique_places.index,
                           numerical_column_name:unique_places.values})
    places[lat_column_name]=np.nan
    places[lon_column_name]=np.nan
    
    return places

In [ ]:
#just join it 
jjit_json = get_jobs_info()
jjit_data = json_normalize(jjit_json)
jjit_data.drop(jjit_data[ jjit_data['country_code'] != 'PL' ].index , inplace=True)

city_column_name = 'city'
grouped_data = group_by_city(jjit_data, city_column_name)
places_map = fill_geo_info(grouped_data, city_column_name)
print_geo_map(places_map)